
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


In [0]:
%run ../../Includes/Classroom-Setup-2L

In [0]:
%sql
-- Creating Loan Details Table
CREATE TABLE IF NOT EXISTS loan_details_silver (
    id INT,
    avg_cc_spending INT,
    total_mortgage_amount INT,
    has_personal_loan BOOLEAN,
    has_securities_account BOOLEAN,
    has_cd_account BOOLEAN,
    is_online_customer BOOLEAN,
    has_credit_card BOOLEAN
);

In [0]:
%sql
-- Deleting Records before Insertion
DELETE FROM loan_details_silver

In [0]:
%sql
-- Transforming and Inserting records into our newly created table
INSERT INTO loan_details_silver
(
    SELECT 
        id,
        CAST(credit_card_average AS FLOAT) * 1000 AS avg_cc_spending,
        CAST(mortgage * 1000 AS FLOAT) AS total_mortgage_amount, 
        has_personal_loan,
        has_securities_account,
        has_cd_account,
        is_online AS is_online_customer,
        has_credit_card
    FROM bank_master_data_bronze
)

#### Setting Up Task Values
We will set up task values to be used in another task. 
- First, we extract the `loan_details_silver` table and save it as a DataFrame. 
- Next, we apply a filter to identify customers who hold both a credit card and a personal loan. 
- These customers are categorized as **risky customers**.
- We will count the number of such customers, and if the count exceeds 100, we will set the `risk_flag` to true.

**NOTE: We are setting up task values for our next task. You will learn more about task values in upcoming lectures before actually using them in the next lab.**

In [0]:
## Getting Dataframe for setting Task Value
loan_details_df = spark.read.table("loan_details_silver")

In [0]:
# Checking for risky customers
## Condition: users with both a credit card and a personal loan

risky_customers_df = loan_details_df.filter(
    (loan_details_df['has_credit_card'] == True) & (loan_details_df['has_personal_loan'] == True)
)

# Getting count of risky customers
risk_count = risky_customers_df.count()
print(f"Number of users with both personal loan and credit card: {risk_count}")

In [0]:
# Decide flag
## We are going to set the flag to True if there are more than 100 customers with both personal loan and credit card
### This task value is going to used by next lab task
status = True if risk_count > 100 else False
dbutils.jobs.taskValues.set(key="risk_flag", value=status)

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>